In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
from groq import Groq
from google.colab import userdata

api = userdata.get('GROQ_API_KEY')

client = Groq(api_key=api)

In [ ]:
system_message = """
You are a cybersecurity expert trained to analyze network flow logs and determine whether the traffic is benign or part of a Distributed Denial of Service (DDoS) attack.

Use the ReAct (Reasoning + Acting) approach:
1. First, REASON through the meaning of the network features such as IPs, ports, protocols, flow duration, packet counts, and byte rates.
2. Then, ACT by concluding whether the flow is BENIGN or DDOS based on patterns, anomalies, or suspicious behavior.

Be concise but clear in your reasoning, and always end with the final classification in the format: "Final Answer: BENIGN" or "Final Answer: DDOS".
"""

In [ ]:
import pandas as pd

pd = pd.read_csv('/content/sample_20_rows.csv')

pd.head(20)

,Source IP,Destination IP,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Flow Bytes/s,Label
0,172.16.0.5,192.168.50.1,842,21484,17,-0.194471,0.325118,-0.145124,-0.416396,drdos_dns
1,172.16.0.5,192.168.50.1,634,45239,17,-0.194038,0.407954,-0.145124,-0.431972,drdos_dns
2,172.16.0.5,192.168.50.1,634,8272,17,-0.194129,0.842839,-0.145124,-0.411765,drdos_dns
3,172.16.0.5,192.168.50.1,634,57621,17,-0.194152,0.842839,-0.145124,-0.410605,drdos_dns
4,192.168.50.6,34.204.21.102,57025,443,6,2.902553,-0.948475,2.009327,-0.491065,benign
5,35.173.44.140,192.168.50.8,443,59168,6,-0.195747,-1.217690,0.027232,-0.491072,benign
6,172.16.0.5,192.168.50.1,634,36811,17,-0.194265,-0.296147,-0.145124,-0.452124,drdos_dns
7,172.16.0.5,192.168.50.1,31043,9846,17,-0.195756,-1.207336,-0.145124,2.362340,drdos_dns
8,172.16.0.5,192.168.50.1,634,16267,17,-0.194227,0.407954,-0.145124,-0.424587,drdos_dns
9,172.16.0.5,192.168.50.1,634,8598,17,-0.194327,0.842839,-0.145124,-0.400777,drdos_dns


In [ ]:
X = pd.drop('Label', axis=1)
y = pd['Label']

In [ ]:
for i in range(20):
    prompt = f"""
Source IP : {X['Source IP'][i]}
Destination IP : {X['Destination IP'][i]}
Source Port : {X['Source Port'][i]}
Destination Port : {X['Destination Port'][i]}
Protocol : {X['Protocol'][i]}
Flow Duration : {X['Flow Duration'][i]}
Total Fwd Packets : {X['Total Fwd Packets'][i]}
Total Backward Packets : {X['Total Backward Packets'][i]}
Flow Bytes/s : {X['Flow Bytes/s'][i]}
"""

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="llama-3.1-8b-instant",
        temperature=0.5,
        max_completion_tokens=1024,
        top_p=0.5,
        stream=False
    )

    print(f"\nResponse for index {i}:")
    print(response.choices[0].message.content)



Response for index 0:
REASON:

1. The Source IP (172.16.0.5) is within a private IP range, which is commonly used in internal networks. This suggests that the traffic is likely originating from within the network.
2. The Destination IP (192.168.50.1) is also within a private IP range, but it's not the same as the source IP. This could indicate that the traffic is being routed through a network device or a proxy.
3. The Source Port (842) is a non-standard port, which could indicate that the traffic is not a typical service or application.
4. The Destination Port (21484) is also a non-standard port, which further suggests that the traffic is not a typical service or application.
5. The Protocol (17) is UDP, which is a connectionless protocol that is often used for streaming or real-time applications.
6. The Flow Duration is negative (-0.1944709240047926), which is unusual and could indicate that the flow is not a typical connection.
7. The Total Fwd Packets (0.3251184533735609) and Tota